In [1]:
import os
import dotenv

dotenv.load_dotenv()

os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# LANGCHAIN_API_KEY from env
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "default" 


In [2]:
from langsmith import traceable

from adalflow import Generator

from abc import ABC, abstractmethod
from typing import Any
from pydantic import BaseModel


class BaseDigestor(BaseModel, ABC):
    llm_client: Any | None
    llm_kwargs: dict | None = {"model": "gpt-4o"}

    _template: str = r"""
    <SYS>{{task_desc_str}}</SYS>
    {# output format #}
    {% if output_format_str %}
    {{output_format_str}}
    {% endif %}
    User: {{input_str}}
    You:"""

    @abstractmethod
    def run(self, text: str) -> str: ...
    
class NewsletterGenerator(BaseDigestor):
    _task_desc_str: str = """
    You need to generate a newsletter based on the bookmarks of the user.
    It should be properly structured and formatted.
    The concepts that fall under the same topic or category should be grouped together.
    Each topic or category need to have a proper headline.
    """

    @traceable(run_type="llm")
    def run(self, text: str) -> str:
        generator = Generator(
            model_client=self.llm_client,
            model_kwargs=self.llm_kwargs,
            prompt_kwargs={"task_desc_str": self._task_desc_str},
            template=self._template,
        )

        return generator(prompt_kwargs={"input_str": text})


In [3]:
from adalflow.components.model_client.openai_client import OpenAIClient
from dotenv import load_dotenv
from langsmith.wrappers import wrap_openai
import openai

load_dotenv()
class OpenAIClientTraceable(OpenAIClient):
    def init_sync_client(self):
        api_key = self._api_key or os.getenv("OPENAI_API_KEY")
        if not api_key:
            raise ValueError("Environment variable OPENAI_API_KEY must be set")
        return wrap_openai(openai.OpenAI(api_key=api_key))
    


smart_newsletter = NewsletterGenerator(llm_client=OpenAIClientTraceable(), llm_kwargs={"model": "gpt-4o", "temperature": 0.5})

In [16]:
output = smart_newsletter.run(""" 
Startup funding news 💸
Sateliot, a Barcelona-based nanosatellite telecommunications company extending connectivity where telecom operators lack coverage, received a €10M investment from Global Portfolio Investments, the family office of the Domínguez family, owners of Mayoral. More info here

Hotelverse, a Palma de Mallorca-based startup that developed a platform allowing hotel guests to select their exact room through digital twins, raised €5M in a Series A round led by GED Conexo Ventures

Homedoctor, a Madrid-based telemedicine provider, raised €4.5M in a funding round led by Q-Impact and Clave Capital. After the transaction, the founders of the company will own 25% of the company

XRF, a startup developing defense AI technology, secured €6M from JME Ventures, Kfund, and GoHub. We wrote about the investment here

Holaglow, a Barcelona-based beauty and wellness platform, raised €1.5M from business angels and family offices

Webel, a Madrid-based on-demand services app, secured €1.4M in funding from Goodwater Capital and Decelera

Danna, a Madrid-based pioneer in producing biomaterials as plastic substitutes, raised €800k from BeAble Capital

Colibid, a Valencia-based mortgage auction platform, closed a €400k funding round and announced the launch of Extracto.io
""")

cache_path: C:\Users\Carlos\AppData\Roaming\adalflow\cache_OpenAIClientTraceable_gpt-4o.db


In [17]:
output

GeneratorOutput(id=None, data='## Startup Funding News 💸\n\n### Telecommunications and Connectivity\n**Sateliot**  \nSateliot, a Barcelona-based nanosatellite telecommunications company, has secured a €10M investment from Global Portfolio Investments, the family office of the Domínguez family, owners of Mayoral. Sateliot aims to extend connectivity to areas where telecom operators lack coverage. [More info here](#)\n\n### Hospitality and Travel Tech\n**Hotelverse**  \nHotelverse, based in Palma de Mallorca, has developed a platform allowing hotel guests to select their exact room through digital twins. The startup raised €5M in a Series A round led by GED Conexo Ventures.\n\n### Telemedicine and Health Tech\n**Homedoctor**  \nHomedoctor, a telemedicine provider from Madrid, raised €4.5M in a funding round led by Q-Impact and Clave Capital. After the transaction, the founders of Homedoctor will retain 25% ownership of the company.\n\n### Defense and AI Technology\n**XRF**  \nXRF, a star